## Building and Deploying a Streamlit App for Maching Learning
In this tutorial, we walk through how you can build and deploy a visual front-end to your machine learning applications. 
We will use [streamlit](https://docs.streamlit.io/library/get-started/main-concepts), which an easy-to-use framework for creating interactive visualizations and applications in Python (this is in contrast to [plotly dash](https://dash.plotly.com) which, although technically python, relies a lot on HTML and CSS knowledge).

### Installing Streamlit and Making a Virtual Environment 
When we make a new front-end application, we want to use a virtual environment. [Pipenv](https://pipenv-fork.readthedocs.io/en/latest/) will be our tool of choice. Pipenv makes it easy to combine both requirements management and virtual environments at the same time.

First, move into the project directory:

`cd my_project`

Then, run:

`pipenv shell` 
to activate the pipenv.

That's all you have to do! 

After that, when you install packages via `pipenv install [package_name]`, they will be automatically added to your `Pipfile.lock`. No need to update a requirements file all the time!

Note: if using pycharm, it can be helpful to use [these steps](https://stackoverflow.com/questions/46251411/how-do-i-properly-setup-pipenv-in-pycharm) to make sure you're using the virtual environment when running your code in pycharm. 